In [1]:
%pip install nltk

In [1]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
import nltk
import numpy as np
import os
import random
import sys

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop